<a href="https://colab.research.google.com/github/bestarch/Redis-Workshops/blob/main/11-LLM_GCP_Workshop/GCP_GenAI_demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd
!pip install --upgrade pip

# Install required libraries
!python3 -m pip -q install redis
!pip install -U langchain-redis gradio
!pip install -U langchain 
!pip install -U langchain-core
!pip install -U langchain-google-vertexai
!pip install jupyterlab_widgets
%pip install -qU pypdf


In [1]:
## Update the 'host' field with the correct Redis host URL
host = 'localhost'
port = 6379
password = 'admin'
requirePass = True


In [3]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)

print(client.ping())
# Clear Redis database (optional)

REDIS_URL = f"redis://:{password}@{host}:{port}"
INDEX_NAME = f"idx_qna"

True


In [ ]:
!wget https://storage.googleapis.com/abhi-data-2024/how_india_shops_online.pdf -O report.pdf


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

#from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader


file = "report.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50, add_start_index=True
)

loader = PyPDFLoader(file)
documents = loader.load()

chunks = text_splitter.split_documents(documents)
#chunked_docs = [doc.page_content for doc in chunks]

In [5]:
chunked_docs = [doc.page_content for doc in chunks]
print(len(chunks))
print(len(chunked_docs))

132
132


In [5]:
#!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2
import os
import subprocess

def check_and_authenticate():
    adc_path = os.path.expanduser("~/.config/gcloud/application_default_credentials.json")
    if os.path.exists(adc_path):
        print("✅ Already authenticated.")
    else:
        print("🔑 Not authenticated. Starting login flow...")
        subprocess.run(["gcloud", "auth", "application-default", "login"])

check_and_authenticate()

✅ Already authenticated.


In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from getpass import getpass
import vertexai
#import google.auth
import vertexai

# input your GCP project ID and region for Vertex AI
PROJECT_ID = 'central-beach-194106'#getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)


#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')



In [8]:
### Following code only works when running the notebook in Google Colab or GCP vertex AI Notebook
'''
from langchain_google_vertexai import VertexAIEmbeddings
from google.colab import auth
from getpass import getpass
import vertexai
import google.auth
import vertexai

# input your GCP project ID and region for Vertex AI
PROJECT_ID = getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

auth.authenticate_user()

#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)
print('Authenticated')

#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')

'''

'\nfrom langchain_google_vertexai import VertexAIEmbeddings\nfrom google.colab import auth\nfrom getpass import getpass\nimport vertexai\nimport google.auth\nimport vertexai\n\n# input your GCP project ID and region for Vertex AI\nPROJECT_ID = getpass("PROJECT_ID:")\nREGION = \'us-central1\' #input("REGION:")\n\nauth.authenticate_user()\n\n#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)\nprint(\'Authenticated\')\n\n#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)\nvertexai.init(project=PROJECT_ID, location=REGION)\n\nprint(f\'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}\')\n\n'

# Create text embeddings with Vertex AI embedding model

Use the Vertex AI API for text embeddings, developed by Google.

Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:


*   Semantic search: Search text ranked by semantic similarity.
*   Recommendation: Return items with text attributes similar to the given text.
*   Classification: Return the class of items whose text attributes are similar to the given text.
*   Clustering: Cluster items whose text attributes are similar to the given text.
*   Outlier Detection: Return items where text attributes are least related to the given text.

The Vertex AI text-embeddings API lets you create a text embedding using Generative AI on Vertex AI. The textembedding-gecko model accepts a maximum of 3,072 input tokens (i.e. words) and outputs 768-dimensional vector embeddings.

In [ ]:
import redis
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain_redis import RedisVectorStore
from langchain_core.documents import Document
import vertexai


embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

def get_vectordb():
    """Create the Redis vectordb."""
    # Load Redis with documents

    vectordb = RedisVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=INDEX_NAME,
    redis_url=REDIS_URL
    )
    return vectordb


redis = get_vectordb()

#embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL)


# Include RAG

We're going to build a complete RAG pipeline from scratch incorporating the following components:

Standard retrieval and chat completion
Dense content representation to improve accuracy
Query re-writing to improve accuracy
Semantic caching to improve performance
Conversational session history to improve personalization

In [ ]:
#@title Invoke Google Vertex LLM using Langchain
# This is where the Langchain brings all the components together in a form of a simple QnA chain
from langchain_google_vertexai import VertexAI
from langchain_redis import RedisSemanticCache
from langchain_core.globals import set_llm_cache
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.outputs import Generation
from langchain import PromptTemplate


cache = RedisSemanticCache(
    redis_url=REDIS_URL,
    embeddings=embeddings,
    distance_threshold=0.05,
    ttl=900,  # 15 min
    prefix="app_cache"
)

# Create LLM
llm = VertexAI(
    model_name="gemini-2.5-pro",
    max_output_tokens=2048,
    temperature=0.5,
    verbose=True,
)

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
#prompt = hub.pull("rlm/rag-prompt")

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

    Question: {question} 
    Context: {context} 
    Answer:""",
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in chunks)

qa_chain = (
    {
        "context": redis.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


/Users/abhishek.srivastava/apps/workspace/redis-ai/chatpdf/venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
questions = [
        "What may be some motivations for shopping online?",
        "What are some preferred methods of payment?",
        "What are some known challenges in shopping online?",
        "How home and kitchen segment is growing?",
        "What are the effects of social media on shopping online?",
        "What are some frequently bought items?"
        ]

In [20]:
%%time
result = None
for question in questions:
   print(f"Question: {question}")
   result = cache.lookup(question, "vertexai/gemini-2.5-pro")
   if result:
      print(f"Cached response: {result[0].text}\n")
   else:
      result = qa_chain.invoke(question)
      print(f"LLM response: {result}\n")
      # Update the cache with the new result
      cache.update(question, "vertexai/gemini-2.5-pro", [Generation(text=result)])


Question: What may be some motivations for shopping online?
Cached response: Motivations for shopping online differ depending on the consumer's location. Urban dwellers often shop online to avoid large crowds in malls and due to a lack of discounts in physical stores. Consumers in other parts of India are motivated by the absence of physical stores for premium brands, product stockouts, and limited access to a wide range of products.

Question: What are some preferred methods of payment?
Cached response: Based on the provided context, payment preferences differ by location. Urban dwellers show a preference for UPI payments, followed by credit/debit cards. Conversely, consumers in the rest of India prefer cash on delivery (CoD) to minimize fraud risk, with Gen Z in these areas also favoring this method.

Question: What are some known challenges in shopping online?
Cached response: Known challenges in online shopping include concerns about payment fraud, the credibility of unfamiliar web

In [ ]:
import gradio as gr
import time

def handle(query):
    start_time = time.time()
    response = qa_chain.invoke(query)
    end_time = time.time()
    processing_time = end_time - start_time
    return response, f"Time: {processing_time:.2f} secs"

iface = gr.Interface(fn=handle, inputs="text", outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="Time")
    ], title="How India shops online?")
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
12:17:16 httpx INFO   HTTP Request: GET http://127.0.0.1:7862/gradio_api/startup-events "HTTP/1.1 200 OK"
12:17:16 httpx INFO   HTTP Request: HEAD http://127.0.0.1:7862/ "HTTP/1.1 200 OK"
12:17:17 httpx INFO   HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
12:17:17 httpx INFO   HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"
* Running on public URL: https://e2f1bf26ca46320657.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
12:17:23 httpx INFO   HTTP Request: HEAD https://e2f1bf26ca46320657.gradio.live "HTTP/1.1 200 OK"


In [60]:
iface.close()

Closing server running on port: 7862
